In [ ]:
!pip install -q sklearn

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
#%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
#variables para predecir o "features"
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

#separo la variable a predecir o "label"
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')


In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []

#variables de categoría
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append( tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))

#variables numéricas
for feature_name in NUMERIC_COLUMNS:
  feature_columns.append( tf.feature_column.numeric_column(feature_name, dtype=tf.float32))



In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):

  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False )

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)


In [ ]:
#desarrollo el modelo
linear_est.train(train_input_fn)

#evaluo el modelo
result = linear_est.evaluate(eval_input_fn)


clear_output()

#veo grado de prediccion
result['accuracy']

In [ ]:
#reviso cada caso, si quiero
result = list(linear_est.predict(eval_input_fn))

print(dfeval.loc[3])
print(result[3]['probabilities'][1])
print(y_eval.loc[3])